In [41]:
import re
import os
from os import path
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

LINUX_COLS = ['i', 'rx_desc', 'rx_bytes', 'tx_desc', 'tx_bytes', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'c0', 'c1', 'c1e', 'c3', 'c6', 'c7', 'joules', 'timestamp']

TIME_CONVERSION_khz = 1./(2899999*1000)
JOULE_CONVERSION = 0.00001526

# 50 100 200 400 500
itrs = ["10", "300"]

dvfss = ["0x0c00",
        "0x1100",
        "0x1d00"]

#iter = 5
## 75

In [47]:
print(24*60/15)

96.0


In [49]:
for itr in itrs:
    for dvfs in dvfss:
        print(f'/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate12500_900000_fbuff-1_itr{itr}_dvfs{dvfs}_rapl135')
        for iter in range(1, 4):
            loc=f'/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate12500_900000_fbuff-1_itr{itr}_dvfs{dvfs}_rapl135_repeat{iter}'
            #print(loc)
            num_interrupts = []
            rxbytes = 0
            txbytes = 0

            tins = 0
            trefcyc = 0
            tllcm = 0
            tjoules = 0
            
            for core in range(0, 16):
                fname = f'{loc}/ITRlogs/linux.flink.dmesg._'+str(core)+"_"+str(loc[len(loc)-1])
                df = pd.read_csv(fname, sep=' ', names=LINUX_COLS)
                rxbytes += df['rx_bytes'].sum()
                txbytes += df['tx_bytes'].sum()

                df_non0j = df[(df['joules']>0) 
                              & (df['instructions'] > 0) 
                              & (df['cycles'] > 0) 
                              & (df['ref_cycles'] > 0) 
                              & (df['llc_miss'] > 0)].copy()
                df_non0j['timestamp'] = df_non0j['timestamp'] - df_non0j['timestamp'].min()
                df_non0j['timestamp'] = df_non0j['timestamp'] * TIME_CONVERSION_khz
                df_non0j['ref_cycles'] = df_non0j['ref_cycles'] * TIME_CONVERSION_khz
                df_non0j['joules'] = df_non0j['joules'] * JOULE_CONVERSION

                ## filters logs to be between 5min - 10 min
                df_non0j = df_non0j[(df_non0j['timestamp'] > 300) & (df_non0j['timestamp'] < 600)]

                tmp = df_non0j[['instructions', 'cycles', 'ref_cycles', 'llc_miss', 'joules', 'c0', 'c1', 'c1e', 'c3', 'c6', 'c7','timestamp']].diff()
                tmp.columns = [f'{c}_diff' for c in tmp.columns]
                df_non0j = pd.concat([df_non0j, tmp], axis=1)
                df_non0j.dropna(inplace=True)
                df.dropna(inplace=True)
                df_non0j = df_non0j[df_non0j['joules_diff'] > 0]

                tins += df_non0j['instructions_diff'].sum()
                trefcyc += df_non0j['ref_cycles_diff'].sum()
                tllcm += df_non0j['llc_miss_diff'].sum()
                cjoules = df_non0j['joules_diff'].sum()
                if core == 0 or core == 1:
                    tjoules += cjoules

                #print(core, round(df_non0j['timestamp'].min(), 2), round(df_non0j['timestamp'].max(), 2), len(df.index))
                #num_interrupts.append(len(df.index))
            print(f'{iter}: {rxbytes} {txbytes} {int(tins)} {round(trefcyc, 2)} {int(tllcm)} {round(tjoules, 2)} Joules')
            print("")
            #plt.scatter(range(0, 16), num_interrupts)
            #plt.title(loc)
            #plt.show()
            #print("")

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate12500_900000_fbuff-1_itr10_dvfs0x0c00_rapl135
1: 1304381547 465251744 65179959454 36.34 57491747 10378.99 Joules

2: 1304027765 459034411 64385405570 35.82 49634589 10363.34 Joules

3: 1304194830 469484376 66218212994 36.11 46238295 10448.28 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate12500_900000_fbuff-1_itr10_dvfs0x1100_rapl135
1: 1303867077 463546634 64431983368 26.87 49468250 10710.26 Joules

2: 1303655482 463924186 65231103118 26.94 48091548 10735.17 Joules

3: 1303887574 453466819 66248646306 27.42 46925326 10693.43 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate12500_900000_fbuff-1_itr10_dvfs0x1d00_rapl135
1: 1304243164 469341012 64785453551 18.03 44734513 11093.5 Joules

2: 1303988783 455838340 64914735525 18.57 51586125 11062.1 Joules

3: 1304133426 469109308 72627535052 19.43 41888836 11146.2 Joules

/mnt/eestreaming-refactor/experiment-scripts/logs/cores16_frate1